# Evrişimsel Sinir Ağları (Convolutional Neural Networks) 

In [1]:
import torch # Pytorch
import torch.nn as nn # Sinir ağı katmanı için 
import torch.optim as optim # Optimizasyon algoritması için
import torchvision # Görüntü işleme için
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
import numpy as np

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

**Problem Tanımı:** CIFAR10 veriseti sınıflandırma problemi

### 1)Veri Setinin Yüklenmesi

In [2]:
# batch_size : Her iterasyonda işlenecek veri sayısını verir
def get_data_loader(batch_size = 64):
    transform = transforms.Compose([
        transforms.ToTensor(), # Görüntüyü tensora çevir
        transforms.Normalize(((0.5,0.5,0.5) ) , (0.5,0.5,0.5)) # Renkli görüntü olduğu için 3 tane oldu
    ])

    # CIFAR10 veri setini indir ve eğitim test veri setini oluşturma
    train_set = torchvision.datasets.CIFAR10(root = "./cifar10_data" , train = True , download = True , transform = transform)
    test_set = torchvision.datasets.CIFAR10(root = "./cifar10_data" , train = False , download = True , transform = transform)

    # Data Loader
    train_loader = torch.utils.data.DataLoader(train_set , batch_size = batch_size , shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_set , batch_size = batch_size , shuffle = False)

    return train_loader , test_loader

In [3]:
train_loader , test_loader = get_data_loader()

### 2)Veri Setinin Görselleştirilmesi

In [4]:
# Görselleştirme
def imshow(img):
    # Veriler normalize etmeden önce geri dönüştürme
    img = img / 2 + 0.5 # Normalize işleminin tersi
    np_img = img.numpy() # Tensor'dan np array'e döndürme 
    plt.imshow(np.transpose(np_img , (1 , 2 , 0))) # Üç kanal için renkleri doğru şekilde gösterme

# Veri kümesinden örnek görselleri almak için fonk.
def get_sample_images(train_loader):

    data_iter = iter(train_loader)
    images , labels = next(data_iter)
    return images , labels

def visualize(n):
    images , labels = get_sample_images(train_loader)
    plt.figure()

    for i in range(n):
        plt.subplot(1 , n , i + 1 )
        imshow(images[i])
        plt.title(f"Label :{labels[i].item()}")
        plt.axis("off")
    plt.show()

In [5]:
# visualize(3)

### 3)Modelin İnşaası

In [10]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(3 , 32 , kernel_size = 3 , padding = 1)
        
        self.relu = nn.ReLU() # Aktivasyon fonk.

        self.pool = nn.MaxPool2d(kernel_size = 2 , stride = 2 ) # 2x2 boyutunda pool katmanı

        self.conv2 = nn.Conv2d(32 , 64 , kernel_size = 3 , padding = 1) # 64 filtreli ikinci conv katmanı

        self.dropout = nn.Dropout(0.2) # Dropout % 20 oranında çalışır

        self.fc1 = nn.Linear(64 * 8 * 8 , 128 ) # Fully connected layer giriş : 4096 , output : 128

        self.fc2 = nn.Linear(128 , 10) # Output katmanı
    
    def forward(self , x):
        # image 3*32*32 => conv (32) -> relu (32) -> pool (16)
        # conv (16) -> relu (16) -> pool (8) -> image 8x8
        # flatten 
        # fc1 -> relu -> dropout
        # fc2 -> output

        x = self.pool(self.relu(self.conv1(x))) # İlk convolution blok
        x = self.pool(self.relu(self.conv2(x))) # İkinci convolution blok
        x = x.view(-1 , 64 * 8 * 8) # Flatten Katmanı
        x = self.dropout(self.relu(self.fc1(x))) # Fully connected katman
        x = self.fc2(x) # Output
        
        return x

In [11]:
model = CNN().to(device)

In [12]:
define_loss_and_optimizer = lambda model :(
    nn.CrossEntropyLoss(), # Multi class classification problem
    optim.SGD(model.parameters() , lr = 0.001 , momentum = 0.9) # Stochastic Gradient Descent
)

### 4)Modelin Eğitimi

In [13]:
def train_model (model , train_loader , criterion , optimizer , epochs = 5 ):
    model.train() # Modeli eğitim moduna alalım
    train_losses = []

    for epoch in range(epochs):
        total_loss = 0 # Toplam Loss değerini saklamak için tutalan değer

        for images , labels in train_loader :
            images , labels = images.to(device) , labels.to(device)

            optimizer.zero_grad() # Gradyanları sıfılama
            outputs = model(images) # Forward pro. (prediction) output = etiket , label , class 
            loss = criterion(outputs , labels) #Loss değerini hesapla
            loss.backward() # Geri yayılım (gradyan hesaplama)
            optimizer.step() # Öğrenme parametre yani ağırlık güncelleme

            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f"Epoch : {epoch + 1 } / {epochs} , Loss :{avg_loss:.5f}")
    # Kayıp (Loss) Grafiği
    # plt.figure()
    # plt.plot(range(1,epochs+1) , train_losses , marker = "o" , linestyle = "-" , label = "Train Loss" )
    # plt.xlabel("Epochs")
    # plt.ylabel("Loss")
    # plt.title("Training Loss")
    # plt.legend()
    # plt.show()

In [14]:
criterion , optimizer = define_loss_and_optimizer(model)
train_model(model , train_loader , criterion , optimizer , 10)

Epoch : 1 / 10 , Loss :2.06700
Epoch : 2 / 10 , Loss :1.73385
Epoch : 3 / 10 , Loss :1.54858
Epoch : 4 / 10 , Loss :1.42344
Epoch : 5 / 10 , Loss :1.34803
Epoch : 6 / 10 , Loss :1.28346
Epoch : 7 / 10 , Loss :1.22839
Epoch : 8 / 10 , Loss :1.17823
Epoch : 9 / 10 , Loss :1.13203
Epoch : 10 / 10 , Loss :1.09397


### 5)Modelin Test Edilmesi

In [15]:
def test_model(model,test_loader):
    model.eval() # Değerlendirme modu
    correct = 0 # Doğru tahmin sayacı
    total = 0 # Toplam Veri sayacı

    with torch.no_grad(): # Gradyan hesaplamasını kapat

        for images , labels in test_loader:
            images , labels = images.to(device) , labels.to(device) # Verileri cihaza taşıma

            outputs = model(images) # Prediction
            _ , predicted = torch.max(outputs , 1) # En yüksek olasılıklı sınıfı seçme
            total += labels.size(0) # Toplam veri sayısı
            correct += (predicted == labels).sum().item() # Doğru tahmin sayısı
        print(f"Test Accuracy :{100 * correct / total} %") # Doğruluk değerinin ekrana yazdırılması

In [16]:
test_model(model , test_loader )

Test Accuracy :62.29 %
